# Setup 

In [ ]:
!pip install -q open_clip_torch transformers

In [ ]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/ #copy api key ---- depend on your directory -- my directory is .../colab/..
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c image-search #download competition dataset

In [ ]:
!unzip /content/image-search.zip

In [ ]:
import numpy as np
import pandas as pd
import os
import open_clip
import cv2
import matplotlib.pyplot as plt
import torch

from numpy.linalg import norm
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from torchvision import transforms
from sklearn.decomposition import PCA

In [ ]:
TEST_FOLDER = '/content/test/images'
QUERY_FOLDER = '/content/queries/queries'

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
class PadToSquare(object):
    def __init__(self):
        return

    def __call__(self, image):
        width, height = image.size
        l, t, r, b = 0, 0, 0, 0

        if width < height:
            lr = height - width
            if lr % 2 == 0:
                l, r = lr // 2, lr // 2
            else:
                l, r = lr // 2 + 1, lr // 2

        elif height < width:
            tb = width - height
            if tb % 2 == 0:
                t, b = tb // 2, tb // 2
            else:
                t, b = tb // 2 + 1, tb // 2

        padding = transforms.Pad([l, t, r, b], padding_mode='edge')
        return padding(image)

In [ ]:
open_clip.list_pretrained()

In [ ]:
model, _, org_transform = open_clip.create_model_and_transforms(
  model_name="ViT-H-14-quickgelu",
  pretrained="metaclip_fullcc",
  device=device
)

transform = transforms.Compose([
    PadToSquare(),
    org_transform
])

transform_gray = transforms.Compose([
    PadToSquare(),
    transforms.Grayscale(3),
    org_transform
])

In [ ]:
transform

In [ ]:
transform_gray

# Get train image embeddings

In [ ]:
data_df = pd.read_csv('/content/sample_submission.csv')

In [ ]:
query_ls = []

for dirname, _, filenames in os.walk(QUERY_FOLDER):
    for filename in filenames:
        if filename.endswith('.jpg'):
            query_ls.append([
                filename,
                dirname.split('/')[-1]
            ])
query_df = pd.DataFrame(query_ls, columns=['filepath', 'class'])

In [ ]:
data_df = data_df.rename(columns={'img_file': 'filepath'})
data_df = data_df.drop('class', axis=1)

In [ ]:
query_df = query_df.drop('class', axis=1)

In [ ]:
Image.open(os.path.join(QUERY_FOLDER, query_df.iloc[1]['filepath'])).convert("RGB")

In [ ]:
class LogoDataset(Dataset):
    def __init__(self, df, transform, folder):
        self.df = df
        self.transform = transform
        self.folder = folder

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        return self.transform(Image.open(os.path.join(self.folder, self.df.iloc[idx]['filepath'])).convert("RGB"))

In [ ]:
data_ds = LogoDataset(data_df, transform, TEST_FOLDER)
data_gray_ds = LogoDataset(data_df, transform_gray, TEST_FOLDER)
query_ds = LogoDataset(query_df, transform, QUERY_FOLDER)

data_loader = DataLoader(data_ds, batch_size=64)
data_gray_loader = DataLoader(data_gray_ds, batch_size=64)
query_loader = DataLoader(query_ds, batch_size=64)

In [ ]:
query_ds[1].shape

In [ ]:
plt.imshow(query_ds[1].permute(1, 2, 0))

In [ ]:
plt.imshow(data_ds[1006].permute(1, 2, 0))

In [ ]:
embs = []
running_vloss = 0

with torch.no_grad():
    for inputs in tqdm(data_loader):
        inputs = inputs.to(device)
        outputs = model.encode_image(inputs)

        embs.append(outputs.detach().cpu().numpy())

data_embs = np.concatenate(embs)
data_embs.shape

In [ ]:
embs = []
running_vloss = 0

with torch.no_grad():
    for inputs in tqdm(data_gray_loader):
        inputs = inputs.to(device)
        outputs = model.encode_image(inputs)

        embs.append(outputs.detach().cpu().numpy())

data_gray_embs = np.concatenate(embs)
data_gray_embs.shape

In [ ]:
embs = []
running_vloss = 0

with torch.no_grad():
    for inputs in tqdm(query_loader):
        inputs = inputs.to(device)
        outputs = model.encode_image(inputs)

        embs.append(outputs.detach().cpu().numpy())

query_embs = np.concatenate(embs)
query_embs.shape

In [ ]:
data_df['emb'] = list(data_embs)
data_gray_df = data_df.copy()
data_gray_df['emb'] = list(data_gray_embs)
query_df['emb'] = list(query_embs)

In [ ]:
data_df

In [ ]:
data_gray_df

In [ ]:
torch.save(query_embs, 'query.pt')
torch.save(data_embs, 'test.pt')
torch.save(data_gray_embs, 'test_gray.pt')

# Evaluate

In [ ]:
data_emb = np.stack(data_df['emb'])
data_gray_emb = np.stack(data_gray_df['emb'])
query_emb = np.stack(query_df['emb'])
print(data_emb.shape)
print(data_gray_emb.shape)
print(query_emb.shape)

In [ ]:
data_df = data_df[['filepath', 'emb']]
data_gray_df = data_gray_df[['filepath', 'emb']]

In [ ]:
data_gray_df

In [ ]:
result = euclidean_distances(query_emb, data_emb)
result_gray = euclidean_distances(query_emb, data_gray_emb)
print(result.shape)
print(result_gray.shape)

In [ ]:
result_gray

In [ ]:
data_df

In [ ]:
data_df['pred'] = '22.jpg'
data_df['min'] = 1000

In [ ]:
data_df

In [ ]:
result

In [ ]:
np.min(result)

In [ ]:
np.max(result)

In [ ]:
np.min(result_gray)

In [ ]:
np.max(result_gray)

#Choosing threshold is different between each models

In [ ]:
thres = 1.1


for i in tqdm(range(len(result))):
    threshold = thres
    pred = query_df.iloc[i]['filepath']
    result_min = np.minimum(result, result_gray)
    if pred == '3.jpg' or pred == '20.jpg':
      threshold+=0.05
    print(f"filename: {pred}, threshold: {threshold}")
    new_min = np.where(result_min[i] < threshold, result_min[i], 1000)
    m = data_df['min']
    data_df['pred'] = data_df['pred'].where(new_min >= m, pred)
    data_df['min'] = data_df['min'].where(new_min >= m, new_min)

In [ ]:
data_df.value_counts('pred')

In [ ]:
data_df

In [ ]:
sub_df = data_df[['filepath', 'pred']].rename(columns={'filepath': 'img_file', 'pred': 'class'})
sub_df['class'] = sub_df['class'].apply(lambda x: int(x.replace('.jpg', '')))

In [ ]:
sub_df

In [ ]:
sub_df.value_counts("class")

In [ ]:
sub_df.to_csv('sub_ViT-H-14-CLIPA-336_datacomp1b_eu_pad_edge.csv', index=False)

# Ensemble
- ลืมชื่อโมเดลครับตั้งมั่วเกิน55

In [ ]:
import pandas as pd
import numpy as np
 
# Read in the prediction CSV files for each model
model1_preds = pd.read_csv("Main.csv")
model2_preds = pd.read_csv(
    "Secondary.csv")
model3_preds = pd.read_csv("tertiary.csv")

# Merge the prediction CSV files into a single DataFrame
merged_preds = pd.merge(model1_preds, model2_preds, on="img_file")
merged_preds = pd.merge(merged_preds, model3_preds, on="img_file")


# Define the function to perform majority voting
def majority_voting(row):
    counts = np.bincount(row)
    return np.argmax(counts)

# Apply the majority voting function to the merged DataFrame
merged_preds["Ensemble_Prediction"] = merged_preds.iloc[:, 1:].apply(majority_voting, axis=1)
merged_preds['class'] = merged_preds["Ensemble_Prediction"]
merged_preds = merged_preds[['img_file','class']]
# Save the final prediction CSV file
merged_preds.to_csv("ensemble_preds.csv", index=False)